In [1]:
import numpy as np
import tensorflow as tf
import gym
import logz
import scipy.signal
import os
import time
import inspect
from multiprocessing import Process

In [2]:
dense = tf.layers.dense

In [3]:
def normalize(data, mean=0.0, std=1.0):
    n_data = (data - np.mean(data)) / (np.std(data) + 1e-8)
    return n_data * (std + 1e-8) + mean

In [4]:
def build_mlp(
        input_placeholder, 
        output_size,
        scope, 
        n_layers=2, 
        size=64, 
        activation=tf.tanh,
        output_activation=None
        ):
    #========================================================================================#
    #                           ----------SECTION 3----------
    # Network building
    #
    # Your code should make a feedforward neural network (also called a multilayer perceptron)
    # with 'n_layers' hidden layers of size 'size' units. 
    # 
    # The output layer should have size 'output_size' and activation 'output_activation'.
    #
    # Hint: use tf.layers.dense
    #========================================================================================#

    with tf.variable_scope(scope):
        
        inputs = input_placeholder
        for _ in range(n_layers):
            inputs = dense(inputs=inputs, units=size, activation=activation)
        outputs = dense(inputs=inputs, units=output_size, activation=output_activation)
    
    return outputs

def pathlength(path):
    return len(path["reward"])

In [5]:
# tf.reset_default_graph()

In [6]:
ph_in = tf.placeholder(dtype=tf.float32, shape=[None, 32], name="test_input")
vn = build_mlp(ph_in, 16, "test_mlp")

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(vn.eval(feed_dict={ph_in: np.random.random([1, 32])}))

[[ 0.84372085 -0.45951545 -0.49178517 -0.1393598   0.01333728 -0.45837387
   0.366988    0.2499969  -0.07101347 -0.13111423  0.2308754  -0.18830796
  -0.13842234  0.26510546 -0.25154674  0.74514043]]


In [8]:
task = "HalfCheetah-v1"

In [9]:
env = gym.make(task)

[2018-06-14 02:09:29,220] Making new env: HalfCheetah-v1


In [10]:
env.reset()

array([ 0.09841633, -0.0401107 , -0.03572379,  0.04084765, -0.06380515,
        0.01330657, -0.09138482,  0.01141045, -0.04391983, -0.05304491,
       -0.11415738, -0.06927074, -0.0966534 , -0.03428902,  0.09363727,
        0.13567018,  0.01698785])

In [11]:
r = 0
for _ in range(1000):
    r += env.step(np.random.random([1, 6]))[1]

In [12]:
r

-174.27512436915669

In [13]:
discrete  = isinstance(env.action_space, gym.spaces.Discrete)

In [59]:
max_path_length = 100
min_timesteps_per_batch=1000
gamma = 0.9
gae_lambda = 0.9
rewards_to_go = True
nn_baseline = True
normalize_advantages = True

In [15]:
ob_dim = env.observation_space.shape[0]

In [16]:
ac_dim = env.action_space.n if discrete else env.action_space.shape[0]

In [17]:
# obseration input placeholder
sy_ob_no = tf.placeholder(shape=[None, ob_dim], name='ob', dtype=tf.float32)
# action input placeholder
sy_nac = tf.placeholder(shape=[None] if discrete else [None, ac_dim], \
                        name="ac", dtype=tf.int32 if discrete else tf.float32)
#advantage input placeholder
sy_adv_n = tf.placeholder(shape=[None], name="adv", dtype=tf.float32)

In [18]:
sess = tf.get_default_session()

In [19]:
#sampel from the output
if discrete:
    sy_logits_na = build_mlp(sy_ob_no, ac_dim, "policy_discrete")
    
    sy_sampled_nac = tf.reshape(tf.multinomial(sy_logits_na, 1), [-1])
    sy_logprob_n = -tf.nn.sparse_softmax_cross_entropy_with_logits(labels=sy_nac, logits=sy_logits_na)
else:
    sy_mean_na = build_mlp(sy_ob_no, ac_dim, "policy_none_discrete")
    sy_logstd = tf.Variable(tf.zeros([1, ac_dim]), name="policy/logstd", dtype=tf.float32)
    sy_std = tf.exp(sy_logstd)
    
    sy_sampled_z = tf.random_normal(tf.shape(sy_mean_na))
    sy_sampled_nac = sy_mean_na + sy_std * sy_sampled_z
    
    sy_z = (sy_nac - sy_mean_na)/sy_std
    sy_logprob_n = -0.5 * tf.reduce_sum(tf.square(sy_z), axis=1)
    
    

In [20]:
learning_rate = 0.001
loss = -tf.reduce_mean(sy_logprob_n * sy_adv_n)
update_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [21]:

if nn_baseline:
    baseline_prediction = tf.squeeze(build_mlp(sy_ob_no, 1, "nn_baseline"))
    sy_target_n = tf.placeholder(shape=[None], name="target", dtype=tf.float32)
    baseline_loss = tf.nn.l2_loss(baseline_prediction - sy_target_n)
    baseline_update_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(baseline_loss)
    

In [22]:
tf_config = tf.ConfigProto(inter_op_parallelism_threads=1, intra_op_parallelism_threads=1)

In [87]:
sess = tf.Session(config=tf_config)
sess.__enter__()
tf.global_variables_initializer().run()

In [88]:
total_timesteps = 0
n_iter = 10000

In [89]:
min_timesteps_per_batch = 1000
max_path_length = 2000

In [90]:
for itr in range(n_iter):
    timesteps_this_batch = 0
    paths = []
    while timesteps_this_batch <= min_timesteps_per_batch:
        ob = env.reset()
        obs, acs, rews = [], [], []
        #execute policy and a the trace
        for _ in range(max_path_length):
            obs.append(ob)
            #get a sampled action
            ac = sess.run(sy_sampled_nac, feed_dict={sy_ob_no: [ob]})[0]
            acs.append(ac)
            
            ob, rew, done, _ =env.step(ac)
            rews.append(rew)           
            if done:
                break
        print(sum(rews), len(rews))
        paths.append({"observation" : np.array(obs), 
                "action" : np.array(acs),
                "reward" : np.array(rews)})
        timesteps_this_batch += pathlength(paths[-1])
        #concatenate all historical paths
        ob_no = np.concatenate([path["observation"] for path in paths])
        ac_nac = np.concatenate([path["action"] for path in paths])
        
        
        q_n = []
        for path in paths:
            q = 0
            q_path =[]
            
            for rew in path["reward"][::-1]:
                q = rew + gamma * q
                q_path.append(q)
            q_path = q_path[::-1]
            
            if not rewards_to_go:
                q_path = [q_path[0]]*len(q_path)
            q_n.extend(q_path)
        
        if nn_baseline:
            #basleine assessment
            b_n = sess.run(baseline_prediction, feed_dict={sy_ob_no: ob_no})
            b_n = normalize(b_n, np.mean(q_n), np.std(q_n))
#             #adv computation
#             adv_n =[]
#             idx = 0
#             for path in paths:
#                 adv = 0
#                 adv_path = []
#                 V_next = 0
#                 idx += len(path["reward"])
                
#                 for rew, V in zip(path["reward"][::-1], b_n[idx-1::-1]):
#                     bellman_error = rew + gamma * V_next -V
#                     adv = bellman_error + gae_lambda * gamma * adv
#                     adv_path.append(adv)

#                     V_next = V
#                 adv_path = adv_path[::-1]
                
#                 if not rewards_to_go:
#                     adv_path = [adv_path[0]] * len(adv_path)
#                 adv_n.extend(adv_path)
            
#             q_n = b_n + adv_n
            adv_n = q_n - b_n
            
        else:
            adv_n = q_n.copy()
        
        if normalize_advantages:
            adv_n = normalize(adv_n)
        
        if nn_baseline:
            q_normalized_n = normalize(q_n)
            sess.run(baseline_update_op, feed_dict={sy_ob_no : ob_no, sy_target_n : q_normalized_n})
        
        sess.run(update_op, feed_dict={sy_ob_no : ob_no, sy_nac : ac_nac, sy_adv_n : adv_n})
        
#         print('{}: return: {}'.format(itr, paths[-1]["reward"].sum()))
        returns = [path["reward"].sum() for path in paths]
        ep_lengths = [pathlength(path) for path in paths]
    
                
                
                
                

-1998.4143814512913 2000
-1979.227086946924 2000
-1838.3224654349158 2000
-1852.2074856290283 2000
-1850.7595559221113 2000
-1741.6521104871397 2000
-1780.2291118382425 2000
-1661.5114147768131 2000
-1617.6801860350333 2000
-1574.5973547106373 2000
-1481.4534660679942 2000
-1626.0590665481823 2000
-1592.8831526519562 2000
-1494.2088282752045 2000
-1484.4716665616725 2000
-1495.8165693281949 2000
-1584.9251499545812 2000
-1452.1636838760492 2000
-1354.5739876152672 2000
-1343.9523595939825 2000
-1504.2996846040521 2000
-1433.5025950663785 2000
-1354.5234179565912 2000
-1298.74250943864 2000
-1451.7476285553164 2000
-1246.4593115888197 2000
-1325.8629623246954 2000
-1334.5680633946374 2000
-1322.7277754495744 2000
-1332.3566014552025 2000
-1389.265590236329 2000
-1425.8936092109743 2000
-1253.1893596910973 2000
-1315.599986917873 2000
-1247.2526441757489 2000
-1251.7592680704638 2000
-1235.4359742380032 2000
-1276.0757609488774 2000
-1323.4671520550519 2000
-1263.0518715797696 2000
-1196

-427.71650474432346 2000
-411.56412659218387 2000
-433.0934738813681 2000
-344.2914019327998 2000
-596.2431380483016 2000
-325.2279765136154 2000
-500.6940388182191 2000
-333.5104830789422 2000
-449.59385326230733 2000
-467.71785443178 2000
-404.75277973371226 2000
-519.7192369128265 2000
-636.3636509318018 2000
-404.8397678445287 2000
-405.5865255862391 2000
-549.8508897780318 2000
-605.3725556274578 2000
-327.06575421296276 2000
-361.85663247553066 2000
-446.1996013054059 2000
-382.85636097376715 2000
-452.83250545940086 2000
-426.4403760730536 2000
-374.80041463943314 2000
-374.64273845298777 2000
-322.5616769892068 2000
-368.57949476043086 2000
-211.54182907214587 2000
-232.88657411048953 2000
-145.05496314582865 2000
-242.3663822503837 2000
-238.7332454766336 2000
-211.24861336161345 2000
-180.22162393866984 2000
-309.7994600841226 2000
-330.5805371486945 2000
-677.59029508335 2000
-278.48967219307355 2000
-280.91199765688015 2000
-308.5954776442227 2000
-298.8388304562951 2000
-1

740.6863930682339 2000
386.46140654040016 2000
449.9915325029719 2000
427.3047530613991 2000
702.8358056949829 2000
773.5910994659687 2000
769.6715347542561 2000
838.392006271207 2000
832.111455227312 2000
599.6516429412344 2000
751.6467123798313 2000
816.5420035596952 2000
743.1926291973658 2000
652.4246963698373 2000
902.5831500223179 2000
911.391917447225 2000
771.9456211380025 2000
363.41148694745993 2000
861.2084606879469 2000
771.4002016671575 2000
457.928195811471 2000
813.5425936653039 2000
663.4923096063778 2000
869.5229320224701 2000
782.5175153003419 2000
418.7088204563639 2000
736.4699921209956 2000
521.1115156431653 2000
-46.86731292966808 2000
793.7069371317642 2000
861.857039160372 2000
114.48454822218063 2000
734.6418836877 2000
756.391726298007 2000
854.3525301608053 2000
806.4685470914551 2000
832.9522200917127 2000
828.6430021318201 2000
989.1587323518359 2000
-38.93942689627311 2000
906.5053569710776 2000
920.3575260093216 2000
912.4443398629566 2000
727.86649357116

1295.6343272933748 2000
928.7554650863315 2000
984.4080199408587 2000
1228.4751096156745 2000
788.3084037384467 2000
1307.040412441113 2000
1281.5921306363437 2000
1150.0153433075732 2000
995.966718751069 2000
1244.023462394201 2000
1565.8917705074844 2000
1189.7027274733778 2000
1275.8886082803547 2000
1339.4770375500746 2000
1537.168394206833 2000
1287.6615176954317 2000
1304.6287249423851 2000
1399.9314077088593 2000
1072.5500398271008 2000
887.8774867585636 2000
925.1159328337371 2000
1290.1563537114473 2000
1122.0621209847388 2000
1364.2882323208744 2000
1192.841971737132 2000
1255.1030891010273 2000
1266.5144179555118 2000
1371.1388434151486 2000
937.2858644517975 2000
1436.7127959066809 2000
1231.0710035774819 2000
1165.8448127150323 2000
1174.2008707271634 2000
1227.6218254321093 2000
1437.366716015972 2000
1380.327951392318 2000
1245.8327096881906 2000
1395.8180865486618 2000
1403.3357686426111 2000
1391.6188148720864 2000
1484.1057278249314 2000
1378.4519499140515 2000
1000.5

1677.2786005379019 2000
2089.176037348058 2000
1769.2578430521544 2000
1874.130325129228 2000
1829.4230703088244 2000
2409.7062251787233 2000
1915.0122442339384 2000
2691.635492965649 2000
1932.9329782121806 2000
2030.1788019173505 2000
1729.589238263541 2000
1900.7500403093816 2000
1836.2891763099788 2000
1925.329294053272 2000
2084.5798673495833 2000
2051.104580800853 2000
2082.932638945189 2000
1898.1170767499664 2000
1718.9124810410578 2000
2052.090318724738 2000
1914.5159983154717 2000
2186.954095007884 2000
1759.2122197916938 2000
1658.8834798257565 2000
2087.9709053886722 2000
2236.518655229562 2000
1763.6868939020812 2000
1974.7010750137974 2000
1743.1702357268334 2000
1494.9451635324717 2000
1811.8635640932555 2000
1813.9822496081379 2000
1571.6434712963558 2000
1718.4238172297578 2000
1781.8841981174746 2000
1996.4541157392753 2000
1834.4912994780475 2000
1998.6768543665069 2000
1560.2034943621138 2000
1831.6933129556828 2000
2209.5873699508725 2000
2180.773628825645 2000
179

...

3056.5048498911333 2000
2644.632298543138 2000
3169.054226188632 2000
3063.50302084929 2000
2950.6140306407165 2000
2806.2325092377873 2000
2980.9880117404477 2000
2917.327670010474 2000
2972.620676897941 2000
3324.661340391817 2000
3210.3077812777115 2000
2763.545195226493 2000
3042.4509454413733 2000
2433.7629912044845 2000
3341.383099595025 2000
2551.444996626398 2000
3168.4208713119906 2000
3384.1978228276807 2000
2604.266941983134 2000
2398.101132857745 2000
3320.494339838554 2000
3683.617457535953 2000
3459.9991641066335 2000
3191.2152238617773 2000
3848.11982429216 2000
3737.1593357918546 2000
2932.9540994051313 2000
3337.159496269034 2000
2824.354179326292 2000
3372.6480219448736 2000
465.1336268308904 2000
3325.1439010158756 2000
3146.350823399762 2000
3098.0538950996643 2000
3108.6001722143988 2000
3277.4786616065294 2000
3545.342764329726 2000
3541.387595699234 2000
3235.098497555384 2000
3031.6681065495804 2000
3214.780664771336 2000
2891.9451795593827 2000
3139.80288083862

3283.035681719093 2000
3734.724063428824 2000
3321.1298146749205 2000
3541.0976762482583 2000
3501.4335904265845 2000
3604.8748444037624 2000
3292.434681673988 2000
3250.731269220978 2000
3307.2319607726013 2000
3342.7771641844993 2000
4493.236882908373 2000
3858.239561581903 2000
3171.53794723484 2000
3267.7300511543053 2000
3476.097463722313 2000
4292.434654761029 2000
4668.360276821545 2000
3959.8231160097484 2000
3274.113866030285 2000
4103.566940229925 2000
3803.3374294607343 2000
3835.489857977118 2000
4352.876898376081 2000
3469.1365228405966 2000
3723.610062886606 2000
3277.0225982015236 2000
4528.894803593584 2000
3358.431195047635 2000
6948.076433250125 2000
3316.9745277272696 2000
3397.5607783179844 2000
5387.696278330915 2000
3913.1125282300427 2000
5493.332066027398 2000
3418.615503140483 2000
7778.211650230172 2000
3492.9033728525387 2000
3561.7492347911984 2000
4078.0064210774312 2000
3535.5106664491823 2000
3217.223725574197 2000
3519.301991798875 2000
4181.432490073578

2965.0713652984487 2000
2795.9316683869074 2000
3599.3486668330474 2000
3371.2890030596263 2000
4348.473622511836 2000
5887.425632778561 2000
3237.9848688604106 2000
3032.825793530871 2000
3973.1967024171813 2000
3382.7181194764007 2000
3180.1829535393367 2000
3837.625464501683 2000
3829.7414936572495 2000
3312.5259885613436 2000
3747.7007267977306 2000
3323.2209335123625 2000
3367.1699633889893 2000
4288.480015343367 2000
3278.7272914493105 2000
3374.526530005208 2000
3626.2101276474204 2000
4118.8121541156215 2000
3399.547095975366 2000
3241.378858254589 2000
3483.963302469784 2000
5043.584753263249 2000
4389.781711840687 2000
5016.285547901356 2000
6221.005280442129 2000
3031.898824259503 2000
3574.4466525866314 2000
5502.330111368665 2000
4021.4292966962234 2000
2956.833190436616 2000
3475.200271565075 2000
3188.1729636553405 2000
3159.5557616108285 2000
3922.02532525364 2000
3660.392465560572 2000
4212.982146320314 2000
5387.516705977565 2000
3415.882747311187 2000
3177.7148386453

3239.950921490007 2000
3215.3984470792525 2000
3117.856291181865 2000
3210.39578925086 2000
3294.4115030761823 2000
3329.7823377076256 2000
3283.760353359871 2000
3299.2838282058156 2000
3238.721402096046 2000
3213.213073621265 2000
3249.3351560969445 2000
3273.900721263049 2000
3376.6625448879095 2000
3269.719401855069 2000
3214.5540044196787 2000
3343.8959516464524 2000
3300.646830977414 2000
3191.985491595519 2000
3375.4654131987922 2000
3163.0743907799533 2000
3058.0170418873786 2000
2740.4434380390167 2000
3049.7461024107265 2000
2907.4591372944824 2000
3072.7860303502985 2000
3210.366933899933 2000
3176.145637064309 2000
3018.8960916520214 2000
3390.9233396521267 2000
2980.363607292516 2000
2949.013542578791 2000
2888.6037224485863 2000
2953.169007883863 2000
3221.640689850033 2000
3023.7196509072305 2000
2925.932311296764 2000
3062.671404523631 2000
3079.0006666944705 2000
3019.7894984820455 2000
2981.6127166477845 2000
2811.031175767101 2000
3111.3895409288584 2000
3149.9859552

3825.0023004319546 2000
3402.939819374075 2000
5271.63320932229 2000
4399.93182864944 2000
3617.55676339487 2000
4130.21847529414 2000
3602.8048817484396 2000
3274.52888939142 2000
3743.5577201821975 2000
3182.516615902018 2000
3492.5633192277255 2000
3316.4146115131953 2000
3170.8506123956263 2000
3398.437169992207 2000
3392.3773210184067 2000
3244.093105333293 2000
3473.2601159665264 2000
3834.1138234430505 2000
4031.916762039577 2000
4790.5882924777925 2000
3886.8185105461803 2000
3377.2977385998543 2000
3345.8158667861694 2000
5313.394220893817 2000
3364.09689891936 2000
3349.307583702015 2000
3239.0932834790115 2000
3891.302353856885 2000
3464.773827357018 2000
3409.2992668658057 2000
6299.83487493262 2000
3864.52595895423 2000
3341.7848864827715 2000
3093.2984583477682 2000
3237.681325182638 2000
3204.2355898665987 2000
3143.079160105991 2000
3138.81891172103 2000
3133.9686647551216 2000
3101.833012147244 2000
3206.5151610832604 2000
6650.965546218814 2000
3217.990155118092 2000


In [ ]:
returns